In [2]:
import torch
import random
import cv2
import models.layers
import addons.trees as trees
from models.vision import HTCNN

def loadData(data_path, data_file):
    output = []
    with open(data_file, 'r') as f:
        for ln in f:
            fields = ln.rstrip('\n').split(',')
            output.append([os.path.join(data_path,fields[0]), int(fields[1])])
            
def loadInBatch(ds, r = 0, batchsize = 16, shuffle=False):
    output_data = None
    output_labels = []
    i = 0
    ndata = len(ds)
    while i<batchsize:
        data_rec = ds[r][0]
        img_data = cv2.imread(ds[r])
        base_label = ds[r][1]
        data_blob = torch.tensor(img_data).float().permute(2,0,1)
        if output_data is None:
            output_data = torch.zeros(batchsize, img_data.shape[2], img_data.shape[0], img_data.shape[1], device=device)
        output_data[i, ...] = img_data
        r += 1
        if r >= ndata:
            r = 0
            if shuffle:
                random.shuffle(ds)
        i += 1
        
def lookupParent(tree, upper_lv=1):
    pass

def train(dataset, label_file):
    nn = HTCNN(label_file, with_aux = True).cuda()
    
def main():
    
    
    #final_y, aux_y = nn(x_)
    #print('--------Final Output-----------')
    #print(final_y)
    #print(final_y.argmax())
    #print(final_y)
    #print('--------Partial Output---------')
    #print(aux_y)
    #print(aux_y[-1].argmax())
    
    #nn.eval()
    #with torch.no_grad():
    #    y = nn(x_)
    #    print(y)
    pass

if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classTree = trees.build_itree('/datasets/vision/cifar100_clean/tree.txt')
    #classTree = trees.build_itree('/datasets/dummy/set1/tree.txt')
    main()

{0: [0, 0], 1: [0, 0], 2: [1, 0], 3: [1, 0], 4: [2, 1], 5: [2, 1], 6: [3, 1], 7: [3, 1]}
